# 9章 潜在顧客を把握するための画像認識

ここでは、カメラから取得した映像を用いて画像認識を行い、
必要な情報を取得するための流れを学ぶことで、
画像認識をビジネス現場で応用するイメージをつかみます。

In [3]:
import cv2
img = cv2.imread('sample_code/chapter_9/img/img01.jpg')
height, width = img.shape[:2]
print(f'画像幅: {str(width)}')
print(f'画像高さ: {str(height)}')
cv2.imshow("img",img)
cv2.waitKey(3000)
cv2.destroyAllWindows()

画像幅: 1920
画像高さ: 1440
